# Energy Data Cleanup

This holds any potentially destructive cleanup -- massaging data to make it fit our expectations.

Cleaups include:

1. Completely null rows are dropped.

In [68]:
# Preliminary setup
import pandas as pd
import zipfile
import os
import matplotlib.pyplot as plt

In [69]:
df = pd.read_pickle('alldata.pickle')

In [70]:
# Remove rows with NaN
df.dropna(how='all',inplace=True)

# Localize index
df.tz_localize(tz='UTC',copy=False,ambiguous='infer')
d = df.tz_convert(tz='Europe/Paris',copy=False)

# Create artificial 'house' column 
subs = df[['sub1','sub2','sub3']].sum(axis=1)
df['house'] = df.active - subs

In [71]:
df.to_pickle("alldata_clean.pickle")